# Filtered state prepartion with GQSP

Ground state preparation is a fundamental task in quantum computations with applications in chemisty, material sciences and physics. 
In this tutorial, we explore how to use [Generalized Quantum Signal Processing](https://journals.aps.org/prxquantum/pdf/10.1103/PRXQuantum.5.020368) for this task:
starting form an inital state, we apply a Gaussian filter to enhance the overlap of the prepared state with the ground state.

In [ ]:
from qrisp import *
from qrisp.gqsp import *
from qrisp.operators import X, Y, Z
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx


def calculate_energy(H, state_prep, state_args=()):
    E = H.expectation_value(state_prep, diagonalisation_method="commuting", precision=0.001)(*state_args)
    return E


def generate_1D_chain_graph(L):
    graph = nx.Graph()
    graph.add_edges_from([(k, (k+1)%L) for k in range(L-1)]) 
    return graph


# Define Heisenberg Hamiltonian with spectrum in [-1,1]
L = 10
G = generate_1D_chain_graph(L)
H = (1/27)*sum((X(i)*X(j) + Y(i)*Y(j) + Z(i)*Z(j)) for i,j in G.edges())
M = nx.maximal_matching(G)
U0 = create_heisenberg_init_function(M)


# Define initial state preparation function
def psi_prep():
    operand = QuantumVariable(H.find_minimal_qubit_amount())
    U0(operand)
    return operand


# Calculate the energy
E = calculate_energy(H, psi_prep)
print(E)


Let us take a closer look at the spectrum of this Hamiltonian and the inital state $\ket{\psi_0}$. 
First, we compute all eigenvalues and the belonging eigenvectors. Secondly, we calculate the fidelities of the initial state $\ket{\psi_0}$ and each eigenstate $\ket{\lambda_k}$.
Next, we find the indices $k$ of the eigenstates $\ket{\lambda_k}$ that have a significant overlap with the initial state, i.e., $F_k=|\langle \lambda_k | \psi_0 \rangle|^2>0.01$.

In [ ]:
H_matrix = H.to_array()
eigvals, eigvecs = np.linalg.eigh(H_matrix)
idx = np.argsort(eigvals)
eigvals_sorted = eigvals[idx].real
print('Eigen energies:', eigvals_sorted)
eigvecs_sorted = eigvecs[:,idx].T

# Initial state
psi0 = psi_prep().qs.statevector_array()
fidelities = np.array([np.abs(eigvecs_sorted[k].conj() @ psi0)**2  for k in range(len(eigvals))])
fidelities_significant = np.where(fidelities>0.01, fidelities,0)
indices = np.argwhere(fidelities_significant)
indices

print(fidelities[indices])
print(eigvals_sorted[indices])

We observe that the inital state already has fidelity $F_0 \approx 0.68$ with the ground state $\ket{\lambda_0}$ with energy $\lambda_0 \approx -0.63$. 
Moreover, the inital state has a significant overlap with only 5 heigher energy eigenstates.

Starting at our inital state $\ket{\psi_0}$, we aim to prepare a state with an even higher overlap with the ground state $\ket{\psi_0}$. 
Therefore, we aim to apply a Gaussian filter centered at $\lambda_0$. This filter is approximated by Chebyshev polynomials of first kind.

In [ ]:
from numpy.polynomial import Chebyshev
from numpy.polynomial.chebyshev import chebval

# Define the Gaussian centered at minimal eigenvalue lambda_0 on [-1, 1]
mu = eigvals_sorted[0]
sigma = 0.05

def gaussian(x):
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2)

# Chebyshev fit
x_nodes = np.cos(np.pi * np.arange(201) / 200)
f_nodes = gaussian(x_nodes)
cheb_fit = Chebyshev.fit(x_nodes, f_nodes, deg=100)
cheb_coeffs = cheb_fit.coef

x_plot = np.linspace(-1, 1, 2000)
f_gaussian = gaussian(x_plot)
f_cheb_10 = chebval(x_plot, cheb_coeffs[:11]) 
f_cheb_20 = chebval(x_plot, cheb_coeffs[:21])  

# Plot
plt.figure(figsize=(10, 5))
plt.plot(x_plot, f_gaussian, color='#20306f', label='Gaussian', linewidth=2)
plt.plot(x_plot, f_cheb_10, color='#6929C4', label='d=10', linestyle='--')
plt.plot(x_plot, f_cheb_20, color='#444444', label='d=20', linestyle='--')
plt.axvline(mu, color='k', linestyle=':', alpha=0.4)
plt.xlabel('x', fontsize=15)
plt.xticks(fontsize=12)
plt.ylabel('f(x)', fontsize=15)
plt.yticks(fontsize=12)
plt.title('Chebyshev approximation of Gaussian filter', fontsize=15)
plt.legend(fontsize=15)
plt.grid()
plt.tight_layout()
plt.show()

In the following, we use GQSP to apply an approximation to the Gaussian filter in Chebyshev basis.

While it is obvious to apply transformations in Fourier basis with GQSP, is may be used in this setting as well:

First, let us recall the notion of a block-encoding:

A block encoding ([Low & Chuang](https://quantum-journal.org/papers/q-2019-07-12-163/pdf/), [Kirby et al.](https://quantum-journal.org/papers/q-2023-05-23-1018/pdf/)) 
of a Hamiltonian $H$ (acting on a Hilbert space $\mathcal H_s$) is a pair of unitaries $(U,G)$, 
where $U$ is the block encoding unitary acting on $\mathcal H_a\otimes H_s$ (for some auxiliary Hilbert space $\mathcal H_a$), 
and $G$ prepares the block encoding state $\ket{G}_a=G\ket{0}_a$ in the auxiliary variable such that $(\bra{G}_a\otimes\mathbb I_s)U(\ket{G_a}\otimes\mathbb I_s)=H$.
Here $\mathbb I_s$ denotes the identity acting on $\mathcal H_s$.

The operator $H$, which is non-unitary in general, is applied as follows:

$$ U\ket{G}_a\ket{\psi}_s = \ket{G}_a H\ket{\psi}_s + \sqrt{1-\|H\ket{\psi}\|^2}\ket{G_{\psi}^{\perp}}_{as},\quad U= \begin{pmatrix}H & *\\* & * \end{pmatrix} $$

where $\ket{G_{\psi}^{\perp}}_{as}$ is a state in $\mathcal H_a\otimes H_s$ orthogonal to $\ket{G}$, i.e., $(\bra{G}_a\otimes\mathbb I_s)\ket{G_{\psi}^{\perp}}_{as}=0$.
Therefore, a block-encoding embeds a not necessarily unitary operator $H$ as a block into a larger unitary operator $U$. In standard form i.e., when $\ket{G}_a=G\ket{0}_a$
 is prepared from the $\ket{0}$ state, $H$ is embedded as the upper left block of the operator $U$.

Given a block-encoding $(U,G)$ of a Hamiltonian $H$, such that $U^2=I$, let

$$ R=(2\ket{G}_a\bra{G}_a-I_a)\otimes I_s $$

be the reflection around $\ket{G}_a$. Then $(RU)^k$ is a block-encoding of $T_k(H)$ where $T_k(x)$ is the $k$-th Chebyshev polynomial of the first kind (see [Kirby et al.](https://quantum-journal.org/papers/q-2023-05-23-1018/pdf/)).

That is, using $RU$ as our unitary for GQSP, we effectively apply a polynomial in Chebyshev basis.

In [ ]:
@RUS
def inner():

    # Pauli block-encoding satisfying U^2=I
    U, state_prep, n = H.pauli_block_encoding()

    # Qubitization step: RU^k is a block-encoding of T_k(H)
    def RU(case, operand):
        U(case, operand)
        reflection(case, state_function=state_prep)

    # Chebyshev approximation of Gaussian filter
    p = cheb_coeffs[:11]

    case = QuantumFloat(n)
    operand = psi_prep()

    with conjugate(state_prep)(case):
        qbl = GQSP([case, operand], RU, p, k=0)

    success_bool = (measure(qbl) == 0) & (measure(case) == 0)
    return success_bool, operand


@jaspify(terminal_sampling=True)
def main(): 

    E = calculate_energy(H, inner)
    return E

# print(main())
# -0.6253936245671972 

Indeed, we achieve a significanlty lower energy -- hence, a higher overlap with the ground state -- for the transformed state after QSP. 